In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import sys 
sys.path.append('..')

import numpy as np 

from hdna import * 

In [3]:
M = Model(standard=True)
M2 = Model(space_dimensionality='2D', 
           sliding=1.5e5, 
           zipping=7.7e7,
           celsius=26  )

In [2]:
M = Model(standard=True)
A = Strand(M, 'AACCAACC')
B = A.complementary()
K = Kinetwork(M, A, B)
K.save_graph('.')

In [27]:
A = Strand(M, 'AAAAAAAAAAAAAAA')
B = Strand(M, 'TTTTTTTTTTTTTTT')
K = Kinetics(M, A, B)
print(K.tdiff_saffdelb())
print(K.pD1*6*np.pi*0.19)
print(f'{1/K.ksphere_sano():.3e}')

3.341532466375093e-08
3.2902480519497757e-06
2.203e+14


In [3]:
def kback(kf, dg):
    return kf * np.exp(dg/(CONST.R*M.kelvin))
def keq(dg):
    return np.exp(-dg/(CONST.R*M.kelvin))

In [4]:
                        #X#
Tl = Strand(M, 'TTGTTAAATATTGATAAG')
R1 = Strand(M, 'AACAATTTAT'        ).invert
R2 = Strand(M,           'AACGATTC').invert
                            #M#

Tr = Strand(M, 'AACAATTTATAACGATTC')
L1 = Strand(M, 'TTGTTAAAT').invert
L2 = Strand(M,          'ATTGATAAG').invert
                        #X# #M#
Tl.length

18

In [5]:
TR1 = Complex(M, Tl, R1, state='duplex', structure='(((((((((.........+.)))))))))')
TR2 = Complex(M, Tl, R2, state='duplex', structure='..........(((.((((+)))).)))') 
TL1 = Complex(M, Tr, L1, state='duplex', structure='(((((((((.........+)))))))))')
TL2 = Complex(M, Tr, L2, state='duplex', structure='..........(((.((((+)))).))).')
O1 = Complex(M, R1.invert, L1, state='duplex', structure='(((((((((.+)))))))))')
O2 = Complex(M, R2.invert, L2, state='duplex', structure='(((.((((+)))).))).')
D = Complex(M, Tl, Tr.invert, state='duplex', structure='(((((((((((((.((((+)))).)))))))))))))')
DUPLEXG = D.G + 7.6 #ABASIC PENALTY

RuntimeError: C++: : "unmatched ( parenthesis" (Types.cc, line 194)


In [17]:
A = Strand(M, 'AACGGGGTATATGGC')
B = A.complementary()
C = Complex(M, A, B.invert, state='duplex')
print(C.G)
print(A.sequence)
print(B.invert.sequence)
KE = keq(C.G)
print(KE)

print(A.sequence+'+'+B.invert.sequence)
print(C.structure)

-23.72928189165215
AACGGGGTATATGGC
GCCATATACCCCGTT
2.174316147827588e+17
AACGGGGTATATGGC+GCCATATACCCCGTT
(((((((((((((((+)))))))))))))))


### 3D Simple



In [ ]:
kz = 2e8
kdl = 4e9
theta = 90
phi   = 110
rho = np.power((theta*phi)/(np.power(360,2)),2)
print(rho)
kn = kdl * rho
kb = kn*np.exp(-0.5/(1.987e-3*(273.15 + 26)))
print(f'{kb:.3e}')
kobs = (kn*kz)/(kz+kn+kb)
print(f'{kobs:.3e}')

0.005835262345679013
1.006e+07
2.000e+07


In [ ]:
kh = kobs

kbtr1 = kback(kh, TR1.G)
kbtr2 = kback(kh, TR2.G)
kbtl1 = kback(kh, TL1.G)
kbtl2 = kback(kh, TL2.G)

kbo1 = kback(kh, O1.G)
kbo2 = kback(kh, O2.G)

kbduplex = kback(kh, DUPLEXG) #abasic penalty

print('kbtr1:',kbtr1)
print('kbtr2:',kbtr2)
print('kbtl1:',kbtl1)
print('kbtl2:',kbtl2)
print('kbo1:',kbo1)
print('kbo2:',kbo2)
print('kbduplex:',kbduplex)

kbtr1: 0.4116495339462537
kbtr2: 119.10838939228304
kbtl1: 0.4116495339462537
kbtl2: 217.73127382045178
kbo1: 0.4116495339462537
kbo2: 217.73127382045178
kbduplex: 0.051567886771378794


### 2D Simple Model 

In [ ]:
#2D SIMPLE MODEL 
k2dl = 1e15*1e-5
rho2 = np.power(theta/360,2)
kn2  = k2dl*rho2
kb2  = kn2*np.exp(-0.5/(1.987e-3*(273.15 + 26)))
kobs2 = (kn2*kz)/(kz+kn2+kb2)
print(f'{kobs2:.3e}')

1.142e+08


In [ ]:
kh = kobs2

kbtr1 = kback(kh, TR1.G)
kbtr2 = kback(kh, TR2.G)
kbtl1 = kback(kh, TL1.G)
kbtl2 = kback(kh, TL2.G)

kbo1 = kback(kh, O1.G)
kbo2 = kback(kh, O2.G)

kbduplex = kback(kh, DUPLEXG) #abasic penalty

print('kbtr1:',kbtr1)
print('kbtr2:',kbtr2)
print('kbtl1:',kbtl1)
print('kbtl2:',kbtl2)
print('kbo1:',kbo1)
print('kbo2:',kbo2)
print('kbduplex:',kbduplex)

kbtr1: 2.350616056023014
kbtr2: 680.1370326562707
kbtl1: 2.350616056023014
kbtl2: 1243.2969940092773
kbo1: 2.350616056023014
kbo2: 1243.2969940092773
kbduplex: 0.29446481199176133


### 3D Zipper Graph Rates

In [ ]:
                        
TlC     = Strand(M, 'TTGTTAAATATTGATAAG')
                             #X#
r1comp  = Strand(M, 'TTGTTAAATA')
R1C     = Strand(M, 'AACAATTTAT'        )
r2comp  = Strand(M,           'TTGATAAG')
R2C     = Strand(M,           'AACTATTC')

                             #X#
TrC     = Strand(M, 'AACAATTTATAACTATTC')

l1comp  = Strand(M, 'AACAATTTA')
L1C     = Strand(M, 'TTGTTAAAT')
l2comp  = Strand(M,          'TAACTATTC')
L2C     = Strand(M,          'ATTGATAAG')
                                 #M#

In [ ]:
TR1C = Complex(M, r1comp, R1C.invert, state='duplex', structure='((((((((((+))))))))))')
TR2C = Complex(M, r2comp, R2C.invert, state='duplex', structure='((((((((+))))))))') 
TL1C = Complex(M, l1comp, L1C.invert, state='duplex', structure='(((((((((+)))))))))')
TL2C = Complex(M, l2comp, L2C.invert, state='duplex', structure='(((((((((+)))))))))')

DC = Complex(M, TlC, TrC.invert, state='duplex', structure='((((((((((((((((((+))))))))))))))))))')

In [ ]:
complexes = {
    'TR1': TR1C,
    'TR2': TR2C,
    'TL1': TL1C,
    'TL2': TL2C,
    'D':   DC
}

complexesoriginal = {
    'TR1': TR1,
    'TR2': TR2,
    'TL1': TL1,
    'TL2': TL2,
    'O1':  O1,
    'O2':  O2,
    'D':   DC
}

In [ ]:
kf = {}
for label, complex in complexes.items():
    print(complex.structure, label)
    S = Simulator(M, complex.s1, complex.s2.invert)
    _ , kf[label] = S.directsimulation()

((((((((((+)))))))))) TR1


0 simulations didn't produce a duplex.
That's 0.0% of simulations
((((((((+)))))))) TR2


0 simulations didn't produce a duplex.
That's 0.0% of simulations
(((((((((+))))))))) TL1


0 simulations didn't produce a duplex.
That's 0.0% of simulations
(((((((((+))))))))) TL2


0 simulations didn't produce a duplex.
That's 0.0% of simulations
((((((((((((((((((+)))))))))))))))))) D


1 simulations didn't produce a duplex.
That's 0.1% of simulations


In [ ]:
complexes['O1'] = complexes['TR1']
complexes['O2'] = complexes['TR2']
kf['O1'] = (kf['TR1']+kf['TL1'])/2
kf['O2'] = (kf['TR2']+kf['TL2'])/2
kf

{'TR1': 1971319.7873039187,
 'TR2': 1816532.4731761925,
 'TL1': 2100300.0822687857,
 'TL2': 1758462.6230980721,
 'D': 1441798.3877100088,
 'O1': 2035809.9347863523,
 'O2': 1787497.5481371323}

In [ ]:
kb = {}
for label, kh in kf.items():
    if label != 'D':
        kb[label] = kback(kh, complexesoriginal[label].G)
    else: 
        kb[label] = kback(kh, DUPLEXG)
kb

{'TR1': 0.04057383735785174,
 'TR2': 10.817997897620197,
 'TL1': 0.043228518523219385,
 'TL2': 19.143234956702567,
 'D': 0.003717450932176441,
 'O1': 0.041901177940535564,
 'O2': 19.459319236613915}